# Week 14 Further Readings

This week, we'll explore some advanced data types supported by [`collections`, a module in the standard library](https://docs.python.org/3/library/collections.html#). Throughout the course, we have encountered multiple container data types like lists, dictionaries, sets, tuples, etc., which store other data type objects within. `collections` include advanced container data types that provide specialized behavior acting as alternatives for general purpose built-in container data types. All the datatypes we'll see below are subclasses of the data type classes we have already worked with and hence support all the methods that the parent class provides. However, they provide additional methods or override methods from the parent class to support a different behavior making data storage and retrieval more convenient than their parent class.


### `deque` or double-ended queue

_Queue_ is a common data structure that mimics any queue, like a line of people waiting at a billing counter. The basic principle of a queue is first-in first-out(FIFO), meaning the person who enters the queue first also leaves the queue first. As a side note, this is in contrast to another common principle, last-in first-out(LIFO) which is the basis for another data structure called _Stack_. This is similar to drawing from a deck of cards, the topmost card is the first to be picked while the bottom card is drawn at the end.

While FIFO is the most common principle associated with a queue, meaning we append to the right end of the queue and remove elements from the left end of the queue, there might be situations where we want the flexibility to append and remove elements from both ends of the queue equally efficiently. A data structure providing this ability is termed as a double-ended queue which can be thought of as a generalization of both queues and stacks. A real-world scenario where double-ended queue is used is for maintaining browser history. The browser needs to append the most recent page you visited at the front in case you want to revisit it. For removing elements, there can be two sub-scenarios: you revisited the page and it needs to be removed from the front of the queue or the browser history has grown so large that some history needs to be removed from the end of the queue.

`deque` (pronounced as deck - **d**ouble **e**nded **q**ueue) provides this exact functionality of adding and removing elements from either end of the queue. Let us look at some examples:

In [1]:
from collections import deque

dq = deque([1, 2, 3]) #initialize a deque

In [2]:
dq.append(4) #insert to the right end - similar to the list.append() function
print(dq)

deque([1, 2, 3, 4])


In [3]:
dq.appendleft(0) #insert to the left end
print(dq)

deque([0, 1, 2, 3, 4])


In [4]:
dq.pop() #remove from right end
print(dq)

deque([0, 1, 2, 3])


In [5]:
dq.popleft() #remove from left end
print(dq)

deque([1, 2, 3])


An optional argument we can pass while initializing the queue is `maxlen` which maintains the `deque` length constant by automatically removing elements from one end if elements are added to the other.

In [6]:
dq_len = deque([1, 2, 3, 4], maxlen = 4)
print(dq_len)

deque([1, 2, 3, 4], maxlen=4)


In [7]:
dq_len.appendleft(0)
print(dq_len)

deque([0, 1, 2, 3], maxlen=4)


In [8]:
dq_len.appendleft(-1)
print(dq_len)

deque([-1, 0, 1, 2], maxlen=4)


The elements 4 and 3 were removed from the right end when 0 and -1 were added to the left end, thus maintaining a constant length of 4. You can imagine this feature being used to maintain the size of your browser history. 

Further, we can use this data structure to create k-mers from a given sequence. We briefly saw k-mers being used in aligners like `minimap2` in the previous week's assignment. We'll implement a code snippet below to print all the 3-mers using deque:

In [9]:
seq = "AGTCTAGCTCTAGC"
print(seq)
k = 3

dq_seq = deque("", maxlen = k) #initialize with first 3 bases

for i in range(0, len(seq)):
    dq_seq.append(seq[i]) #keep only 3 bases at any point
    if len(dq_seq) != k: #fill up the entire deque to get 'k' length k-mer
        continue
    print(" "* (i-(k-1)) + "".join(dq_seq))

AGTCTAGCTCTAGC
AGT
 GTC
  TCT
   CTA
    TAG
     AGC
      GCT
       CTC
        TCT
         CTA
          TAG
           AGC


Further, `deque` is an iterable and mutable data type that supports more methods that work with lists like `extend()`, `insert()` apart from `append()` we saw above and some string methods like `clear()`, `copy()` etc. You can find a list of all supported methods [in the documentation](https://docs.python.org/3/library/collections.html#deque-objects). 

I mentioned about efficiency when introducing double-ended queue. While normal queues only support appending to the right end and removal from the left end, double-ended queues need to perform both operations at both ends. To allow for this, the underlying implementation of `deque` is done using a doubly linked list whereas queues are implemented using a singly linked list. This is an implementation detail and something you don't have to worry about when using `deque` but if you are interested in learning about what the differences are between these two types of linked lists and how this design choice affects the efficiency, see [this tutorial](https://realpython.com/linked-lists-python/).

### namedtuple

Named tuples are a subclass of tuples with the difference that it provides named fields along with indices for accessing elements. These named fields make the code more readable and they can be accessed using the dot notation, like `ntuple.field`. Similar to it's parent class `tuple`, it is immutable. 

We'll look at an example below. We worked quite a bit with BLAST results throughout the semester. You'll probably agree that it is painful to remember which columns correspond to which fields to index them. Let us recreate that using named tuples to see how it makes it more convenient:

In [10]:
from collections import namedtuple

BlastOutput = namedtuple("BlastOutput", ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore", "qlen"])
blast_named_tuple = BlastOutput("sp|P09835|UHPB_ECOLI|311-499", "NC_000913.3", 100.000, 189, 0, 0, 1, 189, 3849206, 3848640, 2.63e-95, 300, 189)

Before we access the elements inside the tuple, let us first understand the above syntax. `namedtuple()` is called the **factory function** which just means that it is a method used to create an object(here the named tuple object). The general syntax for creating a namedtuple which takes two mandatory arguments is as follows:

`namedtuple(typename, field_names)`

1. _typename_ - this is the name of the new tuple subclass(or named tuple class) that will be created. This new class creates tuple-like objects with fields that can be looked up using their named attributes or indices as with normal tuples. In the example above, this is "BlastOutput".
2. _field_names_ - list of field names used to access the elements in the tuple. They can be provided as a list of field names or a single string with each field name separated by whitespace or commas, for example, 'x y' or 'x, y'. The list containing BLAST headers above correspond to this argument.

In [11]:
print(blast_named_tuple.qseqid) #access using field name
print(blast_named_tuple[0]) #access using index

sp|P09835|UHPB_ECOLI|311-499
sp|P09835|UHPB_ECOLI|311-499


Since this is somewhat like a key-value pair, we can also directly pass dictionaries to create named tuples. I'll use a simpler example below to save some typing:

In [12]:
BedLine = namedtuple("BedLine", ["seqid", "start", "stop"])

bed_content = {"seqid": "NC_000913.3", "start": 1, "stop": 189}
bedline = BedLine(**bed_content)

print(bedline)
print(bedline.seqid)

BedLine(seqid='NC_000913.3', start=1, stop=189)
NC_000913.3


You might remember the `**` syntax from the review session a few weeks back. It is just a way to unpack a dictionary into key-value pairs in a function call before passing it to the function.

We can convert existing named tuple instances into dictionaries using `_asdict()` method which returns a new dictionary with field names as keys.

In [13]:
print(blast_named_tuple._asdict())

{'qseqid': 'sp|P09835|UHPB_ECOLI|311-499', 'sseqid': 'NC_000913.3', 'pident': 100.0, 'length': 189, 'mismatch': 0, 'gapopen': 0, 'qstart': 1, 'qend': 189, 'sstart': 3849206, 'send': 3848640, 'evalue': 2.63e-95, 'bitscore': 300, 'qlen': 189}


Even though named tuples are immutable, they have a `_replace()` method to update the field values. However, it is important to keep in mind that this "replace" does not happen in place but creates a new copy of the object under the hood. 

In [14]:
print(bedline)
print(id(bedline)) #prints memory address of the object

BedLine(seqid='NC_000913.3', start=1, stop=189)
139772164152976


In [15]:
bedline = bedline._replace(start=0) #BED file is 0-based!

print(bedline)
print(id(bedline))

BedLine(seqid='NC_000913.3', start=0, stop=189)
139772164153936


Notice the memory location is different even though the variable is still called "bedline".

We can iterate over the elements as in a normal tuple but to get just the field names, we can use the `_fields` attribute.

In [16]:
for cols in bedline:
    print(cols)

NC_000913.3
0
189


In [17]:
for fields in bedline._fields:
    print(fields)

seqid
start
stop


Finally, named tuples provide a convenient way to access the fields using names similar to dictionaries instead of indices, but the big difference is that they are immutable. [The documentation](https://docs.python.org/3/library/collections.html#namedtuple-factory-function-for-tuples-with-named-fields) has more details and examples on the other supported methods and attributes.

### defaultdict

`defaultdict` is a subclass of the `dict` class which returns a dictionary-like object. However, the only difference in terms of its functionality is that it does not throw a `KeyError` if we provide an illegal/missing key. Instead, it **automatically** creates the key and provides it a default value. Prof. Collins introduced it during the review session on Thursday but let us dive deeper into its features.

Apart from handling missing keys automatically, a compelling reason to use `defaultdict` over `dict` is counting the number of elements in an iterable. With `dict`, we first need an `if` conditional to check if the key is present, and if not, manually initialize it to zero and then increment the count on encountering the key again:

In [18]:
seq = 'AGTACTAGACATAATGCCCGTGAGATTA'

count_dict = {}

for base in seq:
    if base not in count_dict:
        count_dict[base] = 0
    count_dict[base] += 1

for k, v in count_dict.items():
    print(f"{k}: {v}")

A: 10
G: 6
T: 7
C: 5


However, `defaultdict` eliminates the need of the `if` conditional because missing keys have a "default" value associated with them(we'll understand the syntax for creating `defaultdict` in a minute). So we can rewrite the above code using `defaultdict` as follows:

In [19]:
from collections import defaultdict

count_ddict = defaultdict(int)
for base in seq:
    count_ddict[base] += 1

for k, v in count_ddict.items():
    print(f"{k}: {v}")

A: 10
G: 6
T: 7
C: 5


Next, let us see how `defaultdict` handles missing keys. Before that, keep in mind that `dict`s also provide ways to supply a default value for missing keys. `setdefault()` and `get()` are two such methods that take a key as an argument. If the key exists, the corresponding value is returned. If not, it assigns the default value(provided as the optional second argument) and returns that value or `None` if the default value is not provided. The difference is that `setdefault()` updates the dictionary but `get()` does not.

In [20]:
some_dict = {1: 'a', 2: 'b', 3: 'c'}
print(some_dict[4])

KeyError: 4

In [21]:
some_dict.setdefault(4, 'd') #for key that doesn't exist
print(some_dict[4])

print(some_dict)

d
{1: 'a', 2: 'b', 3: 'c', 4: 'd'}


In [22]:
some_dict.setdefault(8,) #without default value
print(some_dict[8])

print(some_dict)

None
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 8: None}


In [23]:
some_dict.setdefault(3, 'd') #for key that exists
print(some_dict[3])

print(some_dict)

c
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 8: None}


In [24]:
print(some_dict.get(5, 'e')) #for key that doesn't exist

e


In [25]:
print(some_dict.get(9,)) #without default value

None


In [26]:
print(some_dict.get(3, 'e')) #for key that exists

c


In [27]:
print(some_dict)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 8: None}


`defaultdict` makes it easier to handle this missing-key scenario without any explicit instructions. We can create a `defaultdict` using `defaultdict()` method with one argument called `default_factory` that takes a valid Python callable, writable argument or `None`. If `None` is provided or no arguments are passed, it behaves as a normal dictionary. Let us look at an example:

In [28]:
none_dict = defaultdict() #equivalent to defaultdict(None)

none_dict['a']

KeyError: 'a'

In [29]:
ddict = defaultdict(list)
print(ddict['a'])

[]


In [30]:
ddict['a'].append('aa')
ddict['a'].append('aaa')

print(ddict)
print(ddict['a'])

defaultdict(<class 'list'>, {'a': ['aa', 'aaa']})
['aa', 'aaa']


We are passing `list` as the callable argument, so the `defaultdict` is initialized with an empty list if the key is missing. We can now deal with this dictionary as a normal dictionary. If we initialize the `defaultdict` with a container data type, the default value is a corresponding empty data type and if its initialized with a numerical data type, the default value is 0.

In [31]:
ddict_set = defaultdict(set)
print(ddict_set['a'])

set()


In [32]:
ddict_int = defaultdict(int)
print(ddict_int['a'])

0


Let us look at some applications of `defaultdict` below:

##### Grouping items together
Suppose we have a list of words and want to create a dictionary with the first letter as the key and a list of all words that start with that letter as values:

In [33]:
word_list = ["pen", "laptop", "pencil", "book", "bottle"]

word_ddict = defaultdict(list)
for word in word_list:
    word_ddict[word[0]].append(word)

for k, v in word_ddict.items():
    print(f"{k}: {v}")

p: ['pen', 'pencil']
l: ['laptop']
b: ['book', 'bottle']


We'll look at another variant of grouping items. Say we had some counts data associated with items and we want to aggregate all the counts for each item:

In [34]:
counts_list = [ ("item1", 10), ("item2", 23), ("item3", 18), ("item1", 42), ("item4", 11), ("item2", 93), ("item4", 39),  ("item3", 61), 
               ("item1", 12), ("item3", 4), ("item2", 81)]

sum_ddict = defaultdict(int) #initialized with int

for items in counts_list:
    sum_ddict[items[0]] += items[1]

for k, v in sum_ddict.items():
    print(f"{k}: {v}")

item1: 64
item2: 197
item3: 83
item4: 50


##### Counting the number of occurrences

We already saw an example of this at the beginning of this section. We'll look at another example of counting the number of times each item appears in the counts_list from above:

In [35]:
count_ddict = defaultdict(int)

for items in counts_list:
    count_ddict[items[0]] += 1

for k, v in count_ddict.items():
    print(f"{k}: {v}")

item1: 3
item2: 3
item3: 3
item4: 2


Before we move on, let us build on our example on k-mers. Earlier, we only identified and printed the 3-mers of a sequence using `deque` but in practice, k-mers are used in a lookup table with k-mer as the key and the list of indices where they appear as values. Let us use `defaultdict` to create that lookup table:

In [36]:
seq = "AGTCTAGCTCTAGC"
print(seq)
k = 3

dq_seq = deque("", maxlen = k)
kmer_ddict = defaultdict(list)

for i in range(0, len(seq)):
    dq_seq.append(seq[i]) #keep only 3 bases at any point
    if len(dq_seq) != k: #fill up the entire deque to get 'k' length k-mer
        continue
    kmer = "".join(dq_seq)
    kmer_ddict[kmer].append(i-(k-1))

for k, v in kmer_ddict.items():
    print(f"{k}: {v}")

AGTCTAGCTCTAGC
AGT: [0]
GTC: [1]
TCT: [2, 8]
CTA: [3, 9]
TAG: [4, 10]
AGC: [5, 11]
GCT: [6]
CTC: [7]


### Counter

We frequently encounter problems that require counting the number of ocurrences in a collection of elements. While we saw using `defaultdict` is a more convenient way of counting elements compared to `dict`, `collections` module provides `Counter` class which simplifies this even further. `Counter` is a subclass of `dict` that is specially designed for counting immutable and hashable objects. Let us look at an example:

In [37]:
from collections import Counter

counter = Counter('AGTACTAGACATAATGCCCGTGAGATTA')
print(counter)

Counter({'A': 10, 'T': 7, 'G': 6, 'C': 5})


We can achieve the same thing in a single line of code using `Counter`. This class takes any iterable containing immutable elements and returns the count of elements. We require the elements to be hashable(returns the same hash at all times) and immutable to ensure the count value returned is consistent. Additionally, as its underlying implementation involves a dictionary, the keys need to be immutable. 

In [38]:
counter_nested_list = Counter([[1], [2], [1], [3], [2]]) #list is mutable
print(counter_nested_list)

TypeError: unhashable type: 'list'

In [39]:
counter_list = Counter([1,2,1,3,4,1,2,3,1,1,4,3,1,2]) #ints are immutable
print(counter_list)

Counter({1: 6, 2: 3, 3: 3, 4: 2})


`Counter` objects supports methods like `update()` and `subtract()` to modify the counts if required.

In [40]:
counter_list.update({2:1, 3:-2}) #adds to the existing counts
print(counter_list)

Counter({1: 6, 2: 4, 4: 2, 3: 1})


In [41]:
counter_list.subtract({1:3, 3:-1, 4:2}) #subtracts from existing counts
print(counter_list)

Counter({2: 4, 1: 3, 3: 2, 4: 0})


Note that the order of the elements was changed to keep them in the descending order of counts. 

To get the iterable back from a `Counter` object, we can use `elements()` method.

In [42]:
for ele in counter_list.elements():
    print(ele)

1
1
1
2
2
2
2
3
3


You can refer [the  documentation](https://docs.python.org/3/library/collections.html#counter-objects) to find more such useful methods.

### OrderedDict

`OrderedDict` is a subclass of `dict` that specializes in remembering and rearranging dictionary order. Ordered dictionaries are just like regular dictionaries but have some extra capabilities relating to ordering operations. Historically, the order of insertion of keys in a `dict` was not guaranteed to be conserved which is why `OrderedDict` was implemented. However, from Python version 3.7 onwards, `dict` guarantees the conservation of the order and hence `OrderedDict` has become less popular. But there are several reasons why we may still want to use `OrderedDict`s:

* When using `OrderedDict` instead of a `dict`, it is implied that the order of the keys is somehow important. The [**Zen of Python**](https://peps.python.org/pep-0020/) advocates "Explicit is better than implicit".
* `dict`s are optimized for mapping operations. Tracking insertion orders is a secondary consideration. However, `Orderdicts` are optimized for reordering operations and can handle reordering operations better than `dicts`. We'll see some methods that provide reordering capabilities in a minute.
* When comparing `dict`s using `==`, only the contents are compared and not the order. If the order of the insertions is also crucial for comparison, we should use `OrderedDict`.

Let us look at some examples:

In [43]:
from collections import OrderedDict

odict = OrderedDict()

odict["first"] = 1
odict["second"] = 2
odict["third"] = 3
odict["fourth"] = 4

print(odict)

OrderedDict({'first': 1, 'second': 2, 'third': 3, 'fourth': 4})


We can reorder using `move_to_end()` method and remove items from either end of `OrderedDict` instances using `popitem()`.

In [44]:
odict.move_to_end("first") #move to right end
print(odict)

OrderedDict({'second': 2, 'third': 3, 'fourth': 4, 'first': 1})


In [45]:
odict.move_to_end("first", last = False) #move to left end
print(odict)

OrderedDict({'first': 1, 'second': 2, 'third': 3, 'fourth': 4})


In [46]:
odict.popitem() #removes from right end
print(odict)

OrderedDict({'first': 1, 'second': 2, 'third': 3})


In [47]:
odict.popitem(last = False) #removes from left end
print(odict)

OrderedDict({'second': 2, 'third': 3})


We'll check the equality of `dict` instance versus a `OrderedDict` instance with different orders of elements:

In [48]:
dict(first = 1, second = 2, third = 3) == dict(first = 1, third = 3, second = 2)

True

In [49]:
OrderedDict(first = 1, second = 2, third = 3) == OrderedDict(first = 1, third = 3, second = 2)

False

While `dict` only compares the content of the dictionary, `OrderedDict` also compares the order of elements. If we don't care about the order, we can stick with `dict`.

### ChainMap

`ChainMap` class provides the flexibility to group multiple dictionaries(or mappings) so that we can interact with them as a single unit. To understand that better, we'll into an example first:

In [50]:
from collections import ChainMap

dict1 = {'a': 1, 'b': 2, 'c': 3}
dict2 = {'b': 10, 'c': 20, 'd': 30}

cmap = ChainMap(dict1, dict2)

print(cmap)

ChainMap({'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30})


If we try to access keys from the dictionaries, it works as usual. But when we access overlapping keys in `ChainMap` object, what does that return?

In [51]:
print(dict1['b'])
print(dict2['b'])

2
10


In [52]:
print(cmap['b'])

2


So it returns the value from the dict1 as that is the first argument we passed when creating the `ChainMap()` instance. Let us confirm this by changing the order of the dictionaries:

In [53]:
cmap_rev = ChainMap(dict2, dict1)

print(cmap_rev)
print(cmap_rev['b'])

ChainMap({'b': 10, 'c': 20, 'd': 30}, {'a': 1, 'b': 2, 'c': 3})
10


Now we see it returns a value from dict2. So why do we care about the order and `ChainMap` as a whole? 

When we create multiple dictionaries like these, it is possible to separate the data. In other words, we can have duplicate keys because they are essentially different `dict` objects but by creating a "chain" of these dictionaries, we are creating a single interface to interact with all of the dictionaries. When we look up a key present only in one dictionary, it returns that value. However, if the key is repeated, the value returned depends on the order of dictionaries provided when creating the `ChainMap` object. If the key is not present, it throws `KeyError` as usual. This way, we can enforce some priority on key lookups in the dictionaries to control the context of the value returned. A practical example of this is how Python looks for variables. It first searches the local scope, then the global scope and finally the builtin variables. If the variable is defined in the local scope, it stops searching and returns it but if not, continues searching down the chain.

In [54]:
for k, v in cmap.items(): #when iterating, it goes from last to first dictionary
    print(k, v)

b 2
c 3
d 30
a 1


In [55]:
print(cmap['e']) #key that does not exist

KeyError: 'e'

All the dictionaries in the `ChainMap` object are stored as a list in the `maps` attribute. So we can interact with them as we would with a normal `list`.

In [56]:
print(cmap.maps)

[{'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30}]


In [57]:
print(cmap)

cmap.maps.append({'d': 30, 'e': 40})
print(cmap)

ChainMap({'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30})
ChainMap({'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30}, {'d': 30, 'e': 40})


`ChainMap` has two useful properties to update the order of the dictionaries passed: `new_child()`, a method to add to and `parents`, an attribute to remove a dictionary from the front of the object.

In [58]:
print(cmap)

cmap_new_child = cmap.new_child({'a': 5, 'c': 13})
print(cmap_new_child)

ChainMap({'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30}, {'d': 30, 'e': 40})
ChainMap({'a': 5, 'c': 13}, {'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30}, {'d': 30, 'e': 40})


In [59]:
print(cmap)

cmap_parents = cmap.parents
print(cmap_parents)

ChainMap({'a': 1, 'b': 2, 'c': 3}, {'b': 10, 'c': 20, 'd': 30}, {'d': 30, 'e': 40})
ChainMap({'b': 10, 'c': 20, 'd': 30}, {'d': 30, 'e': 40})


You can find a list of all the methods and attributes supported by `ChainMap` [here](https://docs.python.org/3/library/collections.html#chainmap-objects).